In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

In [ ]:
class FetchFred:
    def __init__(self) -> None:
        self.key = Fred(api_key='d59606a150e09c54fd5158bac863da0d') # Jeg skuler min nøkkel her. Skaff din egen.
        
    def fetch(self, series: dict = {
    # name : series id
    'MTS' : 'CMRMTSPL', # manufacturing and trade sales
    'PILTP' : 'W875RX1', # total personal income less transfer payments
    'ENAP' : 'PAYEMS', # employees on nonagricultural payrolls
    'IPMAN' : 'IPMAN' # industrial production
    }) -> pd.DataFrame:
        
        
        lst = []
        for i in series.keys():
            data = self.key.get_series(series[i])
            data = data.rename(i)
            lst.append(data)
        return pd.concat(lst, axis=1)

In [14]:
import os
from fredapi import Fred
import pandas as pd

class FetchFred:
    """
    Class to fetch data series from the FRED (Federal Reserve Economic Data) API.
    """
    def __init__(self, api_key=None) -> None:
        """
        Initializes the FetchFred object with an API key.
        :param api_key: FRED API key. If None, tries to get from environment variable FRED_API_KEY.
        """
        self.key = Fred(api_key='d59606a150e09c54fd5158bac863da0d')

    def fetch(self, series=None) -> pd.DataFrame:
        """
        Fetches the specified data series from FRED and returns them as a pandas DataFrame.
        :param series: Dictionary mapping series names to FRED series IDs. Uses a default if None.
        :return: DataFrame with each series as a column.
        """
        
        if series is None:
            series = {'MTS': 'CMRMTSPL', 'PILTP': 'W875RX1', 'ENAP': 'PAYEMS', 'IPMAN': 'IPMAN'}
        try:
            lst = [self.key.get_series(series_id).rename(series_name) for series_name, series_id in series.items()]
            return pd.concat(lst, axis=1)
        except Exception as e:
            print(f"Failed to fetch data: {e}")
            return pd.DataFrame()


In [15]:
import os
import pandas as pd
from fredapi import Fred
import pickle
import datetime

class FetchFred:
    """
    Class to fetch data series from the FRED (Federal Reserve Economic Data) API with caching.
    """
    def __init__(self, api_key=None, cache_dir: str ='fred_cache') -> None:
        """
        Initializes the FetchFred object with an API key and cache directory.
        :param api_key: FRED API key. If None, tries to get from environment variable FRED_API_KEY.
        :param cache_dir: Directory to store cached data.
        """
        self.key = Fred(api_key if api_key is not None else os.getenv('FRED_API_KEY'))
        self.cache_dir = cache_dir
        if not os.path.exists(cache_dir):
            os.makedirs(cache_dir)
    
    def _get_cache_path(self, series_id):
        """
        Constructs a file path for a given series ID.
        """
        return os.path.join(self.cache_dir, f"{series_id}.pkl")

    def _is_cache_valid(self, cache_path, max_age_days=1):
        """
        Checks if the cache file exists and is not too old.
        """
        if os.path.exists(cache_path):
            mod_time = os.path.getmtime(cache_path)
            current_time = datetime.datetime.now().timestamp()
            return (current_time - mod_time) / 3600 / 24 <= max_age_days
        return False

    def fetch(self, series=None, max_age_days=1) -> pd.DataFrame:
        """
        Fetches the specified data series from FRED, either from the cache or the API, and returns them as a pandas DataFrame.
        :param series: Dictionary mapping series names to FRED series IDs. Uses a default if None.
        :param max_age_days: Maximum age of the cache in days before it's considered expired.
        :return: DataFrame with each series as a column.
        """
        if series is None:
            series = {'MTS': 'CMRMTSPL', 'PILTP': 'W875RX1', 'ENAP': 'PAYEMS', 'IPMAN': 'IPMAN'}
        
        lst = []
        for series_name, series_id in series.items():
            cache_path = self._get_cache_path(series_id)
            if self._is_cache_valid(cache_path, max_age_days):
                with open(cache_path, 'rb') as f:
                    series_data = pickle.load(f)
            else:
                try:
                    series_data = self.key.get_series(series_id)
                    with open(cache_path, 'wb') as f:
                        pickle.dump(series_data, f)
                except Exception as e:
                    print(f"Failed to fetch data for {series_name}: {e}")
                    continue
            lst.append(series_data.rename(series_name))
        return pd.concat(lst, axis=1)


In [21]:
import os
import pandas as pd
from fredapi import Fred
import pickle
import datetime

class FetchFred:
    def __init__(self, api_key=None, cache_dir='fred_cache') -> None:
        self.key = Fred(api_key if api_key is not None else os.getenv('FRED_API_KEY'))
        self.cache_dir = cache_dir
        if not os.path.exists(cache_dir):
            os.makedirs(cache_dir)

    def _get_cache_path(self, series_id):
        return os.path.join(self.cache_dir, f"{series_id}.pkl")

    def _is_cache_valid(self, cache_path, max_age_days=1):
        if os.path.exists(cache_path):
            mod_time = os.path.getmtime(cache_path)
            current_time = datetime.datetime.now().timestamp()
            return (current_time - mod_time) / 3600 / 24 <= max_age_days
        return False

    def list_cached_series(self):
        """
        Lists all series IDs that are currently cached.
        """
        cached_files = os.listdir(self.cache_dir)
        series_ids = [os.path.splitext(file)[0] for file in cached_files]
        return series_ids

    def check_series_cache_status(self, series_ids, max_age_days=1):
        """
        Checks and reports the cache status of a given series or a list of series.
        :param series_ids: A list of series IDs to check.
        :param max_age_days: Maximum age of the cache in days before it's considered expired.
        :return: A dictionary with series IDs as keys and cache status ('Cached', 'Valid', 'Expired', 'Not Cached') as values.
        """
        status = {}
        for series_id in series_ids:
            cache_path = self._get_cache_path(series_id)
            if os.path.exists(cache_path):
                if self._is_cache_valid(cache_path, max_age_days):
                    status[series_id] = 'Valid'
                else:
                    status[series_id] = 'Expired'
            else:
                status[series_id] = 'Not Cached'
        return status

    def fetch(self, series=None, max_age_days=1) -> pd.DataFrame:
        if series is None:
            series = {'MTS': 'CMRMTSPL', 'PILTP': 'W875RX1', 'ENAP': 'PAYEMS', 'IPMAN': 'IPMAN'}
        
        lst = []
        for series_name, series_id in series.items():
            cache_path = self._get_cache_path(series_id)
            if self._is_cache_valid(cache_path, max_age_days):
                with open(cache_path, 'rb') as f:
                    series_data = pickle.load(f)
            else:
                try:
                    series_data = self.key.get_series(series_id)
                    with open(cache_path, 'wb') as f:
                        pickle.dump(series_data, f)
                except Exception as e:
                    print(f"Failed to fetch data for {series_name}: {e}")
                    continue
            lst.append(series_data.rename(series_name))
        return pd.concat(lst, axis=1)


In [13]:
print(os.getenv('FRED_API_KEY'))

None


In [22]:
fred=FetchFred(api_key='d59606a150e09c54fd5158bac863da0d')

In [28]:
fred.fetch(series={'GDP' : 'GDP', 'MTS': 'CMRMTSPL'})

,GDP,MTS
1946-01-01,NaN,NaN
1946-04-01,NaN,NaN
1946-07-01,NaN,NaN
1946-10-01,NaN,NaN
1947-01-01,243.164,NaN
...,...,...
2023-08-01,NaN,1497717.0
2023-09-01,NaN,1507530.0
2023-10-01,27944.627,1505477.0
2023-11-01,NaN,1514733.0


In [ ]:
# import fredkey
fred = Fred(api_key='d59606a150e09c54fd5158bac863da0d') # Jeg skuler min nøkkel her. Skaff din egen.

In [ ]:
# requesting data used in the Chauvet paper
series = {
   # name : series id
    'MTS' : 'CMRMTSPL', # manufacturing and trade sales
    'PILTP' : 'W875RX1', # total personal income less transfer payments
    'ENAP' : 'PAYEMS', # employees on nonagricultural payrolls
    'IPMAN' : 'IPMAN' # industrial production
}

def gen_dataset(series: dict):
    lst = []
    for i in series.keys():
        data = fred.get_series(series[i])
        data = data.rename(i)
        lst.append(data)
    
    return pd.concat(lst, axis=1)

raw = gen_dataset(series)
raw = raw.dropna()


# preprocessing
df = raw.apply(lambda x: np.log(x))
df = df.diff()
df = df.dropna()

scaler = StandardScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns, index = df.index)

# print